# 10. 텍스트 분석 실습 - 캐글 Mercari Price Suggestion Challenge

p538(557)~

In [1]:
import warnings
warnings.filterwarnings('ignore')

<br>

## 10.1 대회 소개

- Mercari Price Suggestion Challenge는 캐글에서 진행된 Challenge
- 일본의 대형 온라인 쇼핑몰인 Mercari사의 제품에 대해 가격을 예측하는 과제
- 제공되는 데이터 세트는 제품에 대한 여러 속성 및 제품 설명 등의 텍스트 데이터로 구성
- Mercari사는 이러한 데이터를 기반으로 제품 예상 가격을 판매자들에게 제공하고자 한다.
- 이와 같은 프로세스를 구현하기 위해 판매자는 제품명, 브랜드명, 카테고리, 제품 설명 등 다양한 속성 정보를 입력
- ML 모델은 이 속성에 따라 제품의 예측 가격을 판매자에게 자동으로 제공할 수 있다.

<br>

### 10.1.1 데이터 세트 다운로드

- [다운로드 링크](https://www.kaggle.com/c/mercari-price-suggestion-challenge/data)

<br>

### 10.1.2 데이터 세트 속성

- `train_id` : 데이터 ID
- `name` : 제품명
- `item_condition_id` : 판매자가 제공하는 제품 상태
- `category_name` : 카테고리명
- `brand_name` : 브랜드 이름
- `price` : 제품 가격, 예측을 위한 **타깃 속성**
- `shipping` : 배송비 모류 여부
  - `1` : 무료 (판매자가 지불)
  - `0` : 유로 (구매자가 지불)
- `item_description` : 제품에 대한 설명

- `price`가 예측해야 할 타깃 값이다.
- 회귀로 피처를 학습한 뒤 `price`를 예측하는 문제
- Mercari Price Suggestion이 기존 회귀 예제와 다른 점은 `item_description`과 같은 텍스트 형태의 비정형 데이터와 다른 정형 속성을 같이 적용해 휘귀를 수행한다는 점이다.

<br>

## 10.2 데이터 전처리

- 필요한 라이브러리와 `mercari_train.tsv` 데이터를 DataFrame으로 로딩
- 데이터를 간략하게 확인

In [2]:
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

mercari_df = pd.read_csv('./data/Mercari/mercari_train.tsv', sep='\t')
print(mercari_df.shape)
mercari_df.head(3)

(1482535, 8)


,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
